In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

In [3]:
hkd_thb = pd.read_csv("hkdthb.csv")
hkd_thb.drop(hkd_thb.tail(1).index,inplace=True)
hkd_thb["Time"] = pd.to_datetime(hkd_thb["Time"])
hkd_thb.tail()

,Time,Open,High,Low,Last,Change,%Chg,Volume
514,2020-01-24,3.9199,3.9350,3.9166,3.9271,0.0072,+0.18%,1481.0
515,2020-01-23,3.9058,3.9288,3.9020,3.9199,0.0141,+0.36%,1569.0
516,2020-01-22,3.9098,3.9105,3.9008,3.9058,-0.0041,-0.10%,1652.0
517,2020-01-21,3.9022,3.9123,3.8971,3.9099,0.0078,+0.20%,1617.0
518,2020-01-20,3.9135,3.9135,3.8480,3.9021,-0.0046,-0.12%,1692.0


In [4]:
data = [go.Candlestick(x=hkd_thb['Time'],
                       open=hkd_thb["Open"],
                       high=hkd_thb['High'],
                       low=hkd_thb['Low'],
                       close=hkd_thb['Last'])]
layout = go.Layout(
    autosize=True,
    width=1000,
    height=1000,
    yaxis= go.layout.YAxis(autorange = True,fixedrange= False, range=[3, 5])
)
fig = go.Figure(data = data, layout = layout)

fig.show()

In [5]:
#preprocessing
scaler = MinMaxScaler()
scaler.fit(hkd_thb[["Open","High","Low","Last"]])
np_array = scaler.transform(hkd_thb[["Open","High","Low","Last"]])
print(np_array)

[[0.81546708 0.82360923 0.81878354 0.76846537]
 [0.81546708 0.81042838 0.84758497 0.80960582]
 [0.82312404 0.81798798 0.85008945 0.8155377 ]
 ...
 [0.14184533 0.12211669 0.21717352 0.13451971]
 [0.12729709 0.12560574 0.21055456 0.1423651 ]
 [0.14892802 0.12793177 0.12271914 0.12743972]]


In [6]:
preprocessed = pd.DataFrame(np_array,columns=["Open","High","Low","Last"])
preprocessed["Time"] = hkd_thb["Time"]
preprocessed

,Open,High,Low,Last,Time
0,0.815467,0.823609,0.818784,0.768465,2022-01-17
1,0.815467,0.810428,0.847585,0.809606,2022-01-14
2,0.823124,0.817988,0.850089,0.815538,2022-01-13
3,0.824081,0.851328,0.860644,0.823192,2022-01-12
4,0.905054,0.895522,0.869052,0.823192,2022-01-11
...,...,...,...,...,...
514,0.161179,0.169607,0.245438,0.175277,2020-01-24
515,0.134188,0.157589,0.219320,0.161500,2020-01-23
516,0.141845,0.122117,0.217174,0.134520,2020-01-22
517,0.127297,0.125606,0.210555,0.142365,2020-01-21


In [7]:
training_length = int(len(preprocessed)*0.75)
print(training_length)
training = np_array[0:training_length,:]
x_train = []
y_train = []
for i in range(60,training_length):
    x_train.append(training[i-60:i,:])
    y_train.append(training[i,3])
x_train , y_train = torch.tensor(x_train), torch.tensor(y_train)
print(x_train , y_train)

389
tensor([[[0.8155, 0.8236, 0.8188, 0.7685],
         [0.8155, 0.8104, 0.8476, 0.8096],
         [0.8231, 0.8180, 0.8501, 0.8155],
         ...,
         [0.8369, 0.8405, 0.8624, 0.8169],
         [0.8287, 0.8548, 0.8601, 0.8368],
         [0.7896, 0.8217, 0.8254, 0.8287]],

        [[0.8155, 0.8104, 0.8476, 0.8096],
         [0.8231, 0.8180, 0.8501, 0.8155],
         [0.8241, 0.8513, 0.8606, 0.8232],
         ...,
         [0.8287, 0.8548, 0.8601, 0.8368],
         [0.7896, 0.8217, 0.8254, 0.8287],
         [0.8790, 0.8672, 0.7279, 0.7895]],

        [[0.8231, 0.8180, 0.8501, 0.8155],
         [0.8241, 0.8513, 0.8606, 0.8232],
         [0.9051, 0.8955, 0.8691, 0.8232],
         ...,
         [0.7896, 0.8217, 0.8254, 0.8287],
         [0.8790, 0.8672, 0.7279, 0.7895],
         [0.8773, 0.8663, 0.8834, 0.8517]],

        ...,

        [[0.3737, 0.3555, 0.4055, 0.3429],
         [0.3421, 0.3557, 0.4079, 0.3739],
         [0.3214, 0.3373, 0.3855, 0.3419],
         ...,
         [0.4675,

C:\Users\Thanat M\AppData\Local\Temp\ipykernel_17248\1328582673.py:9: UserWarning:

Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)



In [8]:
class LSTM(nn.Module):
    def __init__(self,num_hidden = 50):
        super(LSTM, self).__init__()
        self.num_hidden = num_hidden
        self.lstm1 = nn.LSTMCell(4 , self.num_hidden)
        self.lstm2 = nn.LSTMCell(self.num_hidden , self.num_hidden)
        self.linear = nn.Linear(self.num_hidden, 1)
    
    def forward(self, x, future = 0):
        outputs = []
        num_samples = x.size(0)
        h_t = torch.zeros(num_samples, self.num_hidden, dtype=torch.double)
        c_t = torch.zeros(num_samples, self.num_hidden, dtype=torch.double)
        h_t2 = torch.zeros(num_samples, self.num_hidden, dtype=torch.double)
        c_t2 = torch.zeros(num_samples, self.num_hidden, dtype=torch.double)

        for input_t in input.split(1, dim=1):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.cat(outputs, dim=1)
        return outputs

In [9]:
torch.manual_seed(0)
model = LSTM()
criterion = nn.MSELoss()
optimizer = optim.LBFGS(model.parameters(), lr=0.8)

In [7]:
h_t = torch.zeros(4, 5, dtype=torch.double)
h_t.size(1)

5